# Лабораторная работа №2
## Выполнил студент группы БВТ2005 Тимошин Алексей Витальевич

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
3. [Задание 3](#Задание-№3)

### Задание №1
Реализовать методы поиска в соответствии с заданием. Организовать генерацию начального набора случайных данных. Для всех вариантов добавить реализацию добавления, поиска и удаления элементов. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

In [305]:
import random
import time
import copy
import time
def generate_random_list():
    return [random.randint(0, 100) for i in range(random.randint(10, 50))]
data = generate_random_list()

In [311]:
#Бинарный поиск

data_bin = copy.deepcopy(data)
#O(log(n))
def binary_search(data, required): #просто бинарный поиск, ретурним индекс в уже остортированном массиве, иначе None
    data.sort()
    low = 0
    high = len(data) - 1
    while low <= high:
        mid = (low + high) // 2
        if data[mid] == required:
            return mid
        if data[mid] > required:
            high = mid - 1
        else:
            low = mid + 1
    return None

def binary_search_delete(data, required): #через бинарный поиск находим индекс и pop'аем его из отсортированного массива
    i = binary_search(data, required)
    if i:
        data.pop(i)
    return data

def binary_search_append(data, required): #добавляем в отсортированный массив в нужное место исходный элемент
    data.sort()
    if len(data) == 1:
        if data[0] < required:
            data.append(required)
        else:
            data.insert(0, required)
        return data
    #тут через бинарный поиск определяем куда можем вставить наш элемент
    low = 0
    high = len(data) - 1
    while low <= high:
        mid = (low + high) // 2
        if mid == len(data) - 1:
            data.append(required)
            return data
        if data[mid] <= required and data[mid + 1] >= required:
            data.insert(mid + 1, required)
            return data
        if data[mid] > required:
            high = mid - 1
        else:
            low = mid + 1
print(data_bin)
t1 = time.time()
print(binary_search(data_bin, 15))
t2 = time.time()
print(binary_search_delete(data_bin, 15))
print(binary_search_append(data_bin, 17))
search_time = round((t2 - t1) * 1000000000)
print("--- {0} ns ---".format(search_time))

[9, 65, 50, 7, 65, 29, 74, 42, 48, 75, 59, 11, 74, 65, 56, 41, 15]
3
[7, 9, 11, 29, 41, 42, 48, 50, 56, 59, 65, 65, 65, 74, 74, 75]
[7, 9, 11, 17, 29, 41, 42, 48, 50, 56, 59, 65, 65, 65, 74, 74, 75]
--- 0 ns ---


In [312]:
#Бинарное дерево
data_tree = copy.deepcopy(data)
#O(log(n))
#класс узла, из которых будет состоять бинарное дерево
class Node():
    #инициализация узла
    def __init__(self, data):
        self.left = None
        self.right = None
        self.data = data
    
    #закидываем лист в нужную позицию    
    def append_data(self, data):
        if self.data:
            if data < self.data:
                if self.left is None:
                    self.left = Node(data)
                else:
                    self.left.append_data(data)
            elif data > self.data:
                if self.right is None:
                    self.right = Node(data)
                else:
                    self.right.append_data(data)
        else:
            self.data = data
    
    #печать идёт по принципу LEFT-NODE-RIGHT, в более сложных случаях:
    #это выглядит примерно как LEFT-LEFT - LEFT - LEFT-RIGHT - NODE - RIGHT-LEFT - RIGHT - RIGHT-RIGHT
    #функция уже не используется, т.к. есть прекрасный аналог снизу, но решил сохранить на всякий случай
    def print_Tree(self):
        if self.left:
            self.left.print_Tree()
        print(self.data, end = ' ')
        if self.right:
            self.right.print_Tree()
    
    
    #спасибо зарубежным партнёрам за то, что они абсолютно бесплатно предоставили код для красивого вывода бинарного дерева
    def display(self):
        lines, *_ = self._display_aux()
        for line in lines:
            print(line)
    def _display_aux(self):
        """Returns list of strings, width, height, and horizontal coordinate of the root."""
        # No child.
        if self.right is None and self.left is None:
            line = '%s' % self.data
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle
        # Only left child.
        if self.right is None:
            lines, n, p, x = self.left._display_aux()
            s = '%s' % self.data
            u = len(s)
            first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s
            second_line = x * ' ' + '/' + (n - x - 1 + u) * ' '
            shifted_lines = [line + u * ' ' for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2
        # Only right child.
        if self.left is None:
            lines, n, p, x = self.right._display_aux()
            s = '%s' % self.data
            u = len(s)
            first_line = s + x * '_' + (n - x) * ' '
            second_line = (u + x) * ' ' + '\\' + (n - x - 1) * ' '
            shifted_lines = [u * ' ' + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2
        # Two children.
        left, n, p, x = self.left._display_aux()
        right, m, q, y = self.right._display_aux()
        s = '%s' % self.data
        u = len(s)
        first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s + y * '_' + (m - y) * ' '
        second_line = x * ' ' + '/' + (n - x - 1 + u + y) * ' ' + '\\' + (m - y - 1) * ' '
        if p < q:
            left += [n * ' '] * (q - p)
        elif q < p:
            right += [m * ' '] * (p - q)
        zipped_lines = zip(left, right)
        lines = [first_line, second_line] + [a + u * ' ' + b for a, b in zipped_lines]
        return lines, n + m + u, max(p, q) + 2, n + u // 2

    
    
#цикличное закидывание отдельных листьев и формирование тем самым дерева
def create_Tree(data):
    result = Node(data[0])
    for i in range(1, len(data)):
        result.append_data(data[i])
    return result

#вспомогательная функция для организации рекурсии
def __binary_tree_search(tree, required):
    result = ""
    if tree is None:
        return "Error"
    if tree.data == required:
        return ""
    else:
        if tree.data > required:
            result += "->Left" + __binary_tree_search(tree.left, required)
        else:
            result += "->Right" + __binary_tree_search(tree.right, required)
    if "Error" in result:
        return "Error"
    else:
        return result
#головная функция для бинарного поиска по дереву
#результат работы не индекс (потому что это дерево, какой ещё индекс), а строка, обозначающая координаты на дереве(путь до узла)
def binary_tree_search(tree, required):
    result = "Node"
    if tree.data == required:
        return "Element position is Node"
    if tree.data > required:
        result += "->Left" + __binary_tree_search(tree.left, required)
    else:
        result += "->Right" + __binary_tree_search(tree.right, required)
    if "Error" in result:
        return "Element is not containing in the Tree"
    else:
        return "Element position is " + result

#спасибо гуглу и добрым людям за то, что они собрали мне грязный список (многомерный из многомерных) в одномерный
def flat_list(array, new_list=None):
    if new_list is None:
        new_list = []
    for i in array:
        if type(i) == int:
            new_list.append(i)
        else:
            flat_list(i, new_list)
    return new_list  
#упаковываем всю информацию из дерева или узла в одномерный массив, для дальнейшей пересборки
def pack_in_list(tree):
    result = []
    if tree:
        result.append(tree.data)
    if tree.left:
        result.append(pack_in_list(tree.left))
    if tree.right:
        result.append(pack_in_list(tree.right))
    return flat_list(result)
    
def help_remove(tree):
    #сохраняем левую ветку, правую ветку и ответвления правой ветки
    temp_l = tree.left
    temp_r = tree.right
    temp_r_l = tree.right.left
    temp_r_r = tree.right.right
    tree = temp_r
    if temp_r_l is None and temp_r_r is None: #если справа нет ответвлений
        tree.left = temp_l
        return tree
    elif temp_r_l is None: #если есть ответвление справа
        tree.left = temp_l
        return tree
    elif temp_r_r is None: #если есть ответвление слева
        tree.left = temp_l
        tree.right = temp_r_l
        return tree
    else: #если есть оба ответвления
        tree.right = help_remove(temp_r)
        tree.left = temp_l
        return tree
    
def binary_tree_remove(tree, required):
    if tree is None:
        return tree
    if tree.data == required:
        #tree = help_remove(tree) #если удаляемый элемент является корневым узлом, то сразу вызываем функцию-хелпер
        temp = pack_in_list(tree)
        del temp[0]
        tree = create_Tree(temp)
        return tree
    if tree.left:
        if tree.left.data == required:
            if tree.left.left is None and tree.left.right is None: #если ответвлений нет
                tree.left = None
                return
            elif tree.left.left is None: #если ветвь справа
                tree.left = tree.left.right
                return
            elif tree.left.right is None: #если ветвь слева
                tree.left = tree.left.left
                return
            else: #если обе ветки, то вызываем хелпер для пересборки узла
                #tree.left = help_remove(tree.left)
                temp = pack_in_list(tree.left)
                del temp[0]
                tree.left = create_Tree(temp)
                return
        else:
            binary_tree_remove(tree.left, required)
    if tree.right:
        if tree.right.data == required:
            if tree.right.left is None and tree.right.right is None: #если ответвлений нет
                tree.right = None
                return
            elif tree.right.left is None: #если ветвь справа
                tree.right = tree.right.right
                return
            elif tree.right.right is None: #если ветвь слева
                tree.right = tree.right.left
                return
            else: #если обе ветки, то вызываем хелпер для пересборки узла
                #tree.right = help_remove(tree.right)
                temp = pack_in_list(tree.right)
                del temp[0]
                tree.right = create_Tree(temp)
                return
        else:
            binary_tree_remove(tree.right, required)
    return tree
Tree = create_Tree(data_tree)
Tree.display()
t1 = time.time()
print(binary_tree_search(Tree, 15))
t2 = time.time()
Tree = binary_tree_remove(Tree, 15)
Tree.display()
Tree.append_data(17)
Tree.display()
search_time = round((t2 - t1) * 1000000000)
print("--- {0} ns ---".format(search_time))

 9___________________     
/                    \    
7               ____65_   
               /       \  
        ______50___   74_ 
       /           \     \
    __29___       59    75
   /       \     /        
  11_     42_   56        
     \   /   \            
    15  41  48            
Element position is Node->Right->Left->Left->Left->Right
 9_________________     
/                  \    
7             ____65_   
             /       \  
      ______50___   74_ 
     /           \     \
    29___       59    75
   /     \     /        
  11    42_   56        
       /   \            
      41  48            
 9___________________     
/                    \    
7               ____65_   
               /       \  
        ______50___   74_ 
       /           \     \
    __29___       59    75
   /       \     /        
  11_     42_   56        
     \   /   \            
    17  41  48            
--- 0 ns ---


In [313]:
#Метод Фибоначчи


fibonacci_data = copy.deepcopy(data)
#O(ln(n))
def fibonacci_search(data, required):
    
    data.sort()
    print(data)
    if len(data) < 4:
        for i in range(len(data)):
            if data[i] == required:
                return i
        return None
    fb2 = 0
    fb1 = 1
    fb = fb2 + fb1
    while fb < len(data):
        fb2 = fb1
        fb1 = fb
        fb = fb2 + fb1
    m = (fb1 + fb2) - len(data) - 1
    i = fb - m
    p = fb1
    q = fb2
    while True:
        if i < 0:
            if p == 1:
                return None
            else:
                i = i + q
                p = p - q
                q = q - p
        elif i >= len(data):
            if q == 0:
                return None
            else:
                i = i - q
                p, q = q, p - q
        if p < 0 or q < 0: #если вспомогательные ушли в минус, значит всё поломалось и этого элемента тут точно нет
            return None
        if required < data[i]:
            i = i - q
            p, q = q, p - q
        elif required > data[i]:
            i = i + q
            p = p - q
            q = q - p
        else:
            return i

def fibonacci_search_delete(data, required):
    data.sort()
    i = fibonacci_search(data, required)
    if i is not None:
        data.pop(i)
    return data

def fibonacci_search_append(data, required):
    data.sort()
    if len(data) < 4:
        data.append(required)
        data.sort()
        return data
    fb2 = 0
    fb1 = 1
    fb = fb2 + fb1
    while fb < len(data):
        fb2 = fb1
        fb1 = fb
        fb = fb2 + fb1
    m = (fb1 + fb2) - len(data) - 1
    i = fb - m
    p = fb1
    q = fb2
    while True:
        if i < 0:
            if p == 1:
                data.insert(0, required)
                break
            else:
                i = i + q
                p = p - q
                q = q - p
        elif i >= len(data):
            if q == 0:
                data.insert(i, required)
                break
            else:
                i = i - q
                p, q = q, p - q
        if required < data[i]:
            if required <= data[i] and required >= data[i - 1]:
                data.insert(i, required)
                break
            else:
                i = i - q
                p, q = q, p - q
        elif required > data[i]:
            if required >= data[i] and required <= data[i + 1]:
                data.insert(i + 1, required)
                break
            else:
                i = i + q
                p = p - q
                q = q - p
        else:
            data.insert(i, required)
            break
    return data
t1 = time.time()       
print(fibonacci_search(fibonacci_data, 15))
t2 = time.time()
print(fibonacci_search_delete(fibonacci_data, 15))
print(fibonacci_search_append(fibonacci_data, 17))
search_time = round((t2 - t1) * 1000000000)
print("--- {0} ns ---".format(search_time))

[7, 9, 11, 15, 29, 41, 42, 48, 50, 56, 59, 65, 65, 65, 74, 74, 75]
3
[7, 9, 11, 15, 29, 41, 42, 48, 50, 56, 59, 65, 65, 65, 74, 74, 75]
[7, 9, 11, 29, 41, 42, 48, 50, 56, 59, 65, 65, 65, 74, 74, 75]
[7, 9, 11, 17, 29, 41, 42, 48, 50, 56, 59, 65, 65, 65, 74, 74, 75]
--- 0 ns ---


In [341]:
#Интерполяционный
#в среднем log(log(N)), но в худшем случае O(N)
interpol_data = copy.deepcopy(data)

def interpolation_search(data, required):
    data.sort()
    l = 0
    r = len(data) - 1
    while data[l] < required and data[r] > required:
        if data[l] == data[r]:
            break
        i = (((required - data[l]) * (l - r)) // (data[l] - data[r])) + l
        if data[i] == required:
            return i
        if data[i] > required:
            r = i - 1
            continue
        if data[i] < required:
            l = i + 1
    if data[l] == required:
        return l
    if data[r] == required:
        return r
    return None

def interpolation_search_delete(data, required):
    i = interpolation_search(data, required)
    if i:
        data.pop(i)
    return data

def interpolation_search_append(data, required):
    data.sort()
    l = 0
    r = len(data) - 1
    while data[l] < required and data[r] > required:
        if data[l] == data[r]:
            break
        i = (((required - data[l]) * (l - r)) // (data[l] - data[r])) + l
        if data[i] == required:
            data.insert(i, required)
            return data
        if data[i] > required:
            r = i - 1
            continue
        if data[i] < required:
            l = i + 1
    if data[l] > required:
        data.insert(l, required)
    elif data[r] < required:
        data.insert(r + 1, required)
    return data

t1 = time.time()
print(interpolation_search(interpol_data, 15))
t2 = time.time()
print(interpolation_search_delete(interpol_data, 15))
print(interpolation_search_append(interpol_data, 28))
search_time = round((t2 - t1) * 1000000000)
print("--- {0} ns ---".format(search_time))

3
[7, 9, 11, 29, 41, 42, 48, 50, 56, 59, 65, 65, 65, 74, 74, 75]
[7, 9, 11, 28, 29, 41, 42, 48, 50, 56, 59, 65, 65, 65, 74, 74, 75]
--- 0 ns ---


In [315]:
#используя встроенную функцию index
print(data)
t1 = time.time()
print(data.index(15))
t2 = time.time()

search_time = round((t2 - t1) * 1000000000)
print("--- {0} ns ---".format(search_time))

[9, 65, 50, 7, 65, 29, 74, 42, 48, 75, 59, 11, 74, 65, 56, 41, 15]
16
--- 0 ns ---


### Задание №2
Написать соответствующие решения для заданных методов.

In [ ]:
#Простое рехэширование

In [ ]:
#Рехэширование с помощью псевдослучайных чисел


In [ ]:
#Метод цепочек

### Задание №3
Расставить на стандартной 64-клеточной шахматной доске 8 ферзей так, чтобы ни один из них не находился под боем другого». Подразумевается, что ферзь бьёт все клетки, расположенные по вертикалям, горизонталям и обеим диагоналям
Написать программу,  которая находит хотя бы один способ решения задач.


In [5]:
field = [[0 for i in range(8)] for j in range(8)]

# обозначения полей:
# 0 - пустая клетка(нет ни ферзя не удара от него)
# >0 - число, означающее сколько ферзей бьёт эту клетку
# -1 - на этой клетке стоит ферзь

def set_queen(i, j):
    for k in range(8):
        field[k][j] += 1
        field[i][k] += 1
        
        if 0 <= i + j - k <= 7:
            field[i + j - k][k] += 1
        if 0 <= i - j + k <= 7:
            field[i - j + k][k] += 1
    field[i][j] = -1

def delete_queen(i, j):
    for k in range(8):
        field[k][j] -= 1
        field[i][k] -= 1
        
        if 0 <= i + j - k <= 7:
            field[i + j - k][k] -= 1
        if 0 <= i - j + k <= 7:
            field[i - j + k][k] -= 1
    field[i][j] = 0
    
def print_answer():
    key = "ABCDEFGH"
    result = []
    for i in range(8):
        for j in range(8):
            if field[i][j] == -1:
                result.append(key[j] + str(i + 1))
    print(' '.join(result))

def chess_queens(i):
    for j in range(8):
        if field[i][j] == 0:
            set_queen(i, j)
            if i == 7:
                print_answer()
            else:
                chess_queens(i + 1)
            delete_queen(i, j)

    
chess_queens(0)

A1 E2 H3 F4 C5 G6 B7 D8
A1 F2 H3 C4 G5 D6 B7 E8
A1 G2 D3 F4 H5 B6 E7 C8
A1 G2 E3 H4 B5 D6 F7 C8
B1 D2 F3 H4 C5 A6 G7 E8
B1 E2 G3 A4 C5 H6 F7 D8
B1 E2 G3 D4 A5 H6 F7 C8
B1 F2 A3 G4 D5 H6 C7 E8
B1 F2 H3 C4 A5 D6 G7 E8
B1 G2 C3 F4 H5 E6 A7 D8
B1 G2 E3 H4 A5 D6 F7 C8
B1 H2 F3 A4 C5 E6 G7 D8
C1 A2 G3 E4 H5 B6 D7 F8
C1 E2 B3 H4 A5 G6 D7 F8
C1 E2 B3 H4 F5 D6 G7 A8
C1 E2 G3 A4 D5 B6 H7 F8
C1 E2 H3 D4 A5 G6 B7 F8
C1 F2 B3 E4 H5 A6 G7 D8
C1 F2 B3 G4 A5 D6 H7 E8
C1 F2 B3 G4 E5 A6 H7 D8
C1 F2 D3 A4 H5 E6 G7 B8
C1 F2 D3 B4 H5 E6 G7 A8
C1 F2 H3 A4 D5 G6 E7 B8
C1 F2 H3 A4 E5 G6 B7 D8
C1 F2 H3 B4 D5 A6 G7 E8
C1 G2 B3 H4 E5 A6 D7 F8
C1 G2 B3 H4 F5 D6 A7 E8
C1 H2 D3 G4 A5 F6 B7 E8
D1 A2 E3 H4 B5 G6 C7 F8
D1 A2 E3 H4 F5 C6 G7 B8
D1 B2 E3 H4 F5 A6 C7 G8
D1 B2 G3 C4 F5 H6 A7 E8
D1 B2 G3 C4 F5 H6 E7 A8
D1 B2 G3 E4 A5 H6 F7 C8
D1 B2 H3 E4 G5 A6 C7 F8
D1 B2 H3 F4 A5 C6 E7 G8
D1 F2 A3 E4 B5 H6 C7 G8
D1 F2 H3 B4 G5 A6 C7 E8
D1 F2 H3 C4 A5 G6 E7 B8
D1 G2 A3 H4 E5 B6 F7 C8
D1 G2 C3 H4 B5 E6 A7 F8
D1 G2 E3 B4 F5 A